Day 1

In [96]:
import heapq
import time
import random
import pandas as pd
import numpy as np

In [25]:
class Order:
    def __init__(self, order_id, side, price, qty, timestamp):
        self.id = order_id
        self.side = side        # 'buy' or 'sell'
        self.price = price
        self.qty = qty
        self.ts = timestamp

Heap-based OrderBook

Python lists require resorting on every insertion, leading to O(n log n) time, which is infeasible for high-frequency systems. Priority queues like heaps support O(log n) insertion and constant-time access to best prices. Since Python’s heapq is a min-heap, bid prices are inverted to simulate max-heap behavior.

In [26]:
class OrderBook:
    def __init__(self):
        self.bids = []   # max-heap using negative prices
        self.asks = []   # min-heap
        self.trades = []
        self.tape = []

    def add_order(self, order):
        if order.side == 'buy':
            # negative price → max-heap behavior
            heapq.heappush(self.bids, (-order.price, order.ts, order))
        else:
            heapq.heappush(self.asks, (order.price, order.ts, order))

        self.match()

    def match(self):
        while self.bids and self.asks:
            buy = self.bids[0][2]
            sell = self.asks[0][2]

            if buy.price < sell.price:
                break

            traded_qty = min(buy.qty, sell.qty)

            self.trades.append(
                (buy.id, sell.id, sell.price, traded_qty)
            )

            self.tape.append({
                "time": time.time(),
                "price": sell.price,
                "qty": traded_qty
            })


            buy.qty -= traded_qty
            sell.qty -= traded_qty

            if buy.qty == 0:
                heapq.heappop(self.bids)
            if sell.qty == 0:
                heapq.heappop(self.asks)

    def print_top5(self):
        print("\nBUY SIDE")
        for _, _, o in heapq.nsmallest(5, self.bids):
          print(o.price, o.qty)

        print("SELL SIDE")
        for _, _, o in heapq.nsmallest(5, self.asks):
          print(o.price, o.qty)

    def best_bid(self):
        return self.bids[0][2].price if self.bids else None

    def best_ask(self):
        return self.asks[0][2].price if self.asks else None

    def mid_price(self):
        if self.best_bid() is None or self.best_ask() is None:
            return None
        return (self.best_bid() + self.best_ask()) / 2

I refactored the order book to use heap-based priority queues. Bid prices were inverted to simulate a max-heap using Python’s min-heap implementation. This ensures O(log n) insertion while preserving price–time priority

Measuring Time: List vs Heap

In [5]:
def benchmark_list(n=10_000):
    orders = []
    start = time.time()
    for i in range(n):
        price = random.randint(90, 110)
        orders.append((price, i))
        orders.sort(key=lambda x: (-x[0], x[1]))
    return time.time() - start


def benchmark_heap(n=10_000):
    heap = []
    start = time.time()
    for i in range(n):
        price = random.randint(90, 110)
        heapq.heappush(heap, (-price, i))
    return time.time() - start


t_list = benchmark_list()
t_heap = benchmark_heap()

print(f"List insert time: {t_list:.4f} sec")
print(f"Heap insert time: {t_heap:.4f} sec")

List insert time: 13.8341 sec
Heap insert time: 0.0072 sec


I compared order insertion using a sorted list versus a heap-based priority queue.
List insertion required resorting the entire structure (O(n log n)), while heap insertion required only O(log n) operations.
For 10,000 orders, heap-based insertion was significantly faster, demonstrating why exchanges avoid naive list sorting.

Day 2

In [59]:
book = OrderBook()

In [60]:
# submit the ask ladder
asks = [
    (101, 10),
    (102, 20),
    (103, 30),
]

order_id = 1
for price, qty in asks:
    order = Order(
        order_id=order_id,
        side="sell",
        price=price,
        qty=qty,
        timestamp=order_id
    )
    book.add_order(order)
    order_id += 1

In [61]:
# submit the market buy (qty = 60)
# note that Market buy = price = +∞
market_buy = Order(
    order_id=order_id,
    side="buy",
    price=float("inf"),   # market order
    qty=60,
    timestamp=order_id
)

book.add_order(market_buy)

In [62]:
# VALIDATION
assert len(book.trades) == 3, "Expected 3 trades"
expected = [
    (101, 10),
    (102, 20),
    (103, 30),
]

for trade, (price, qty) in zip(book.trades, expected):
    _, _, trade_price, trade_qty = trade
    assert trade_price == price, f"Expected price {price}, got {trade_price}"
    assert trade_qty == qty, f"Expected qty {qty}, got {trade_qty}"
assert len(book.asks) == 0, "Ask book should be empty"
assert len(book.bids) == 0, "Bid book should be empty after market buy"
print("✅ Validation test PASSED: Market buy cleared entire ask book")

✅ Validation test PASSED: Market buy cleared entire ask book


I validated the matching engine using a market buy order that fully consumed a multi-level ask ladder, confirming correct book walking, partial fills, and price–time priority.

Day 3

Define an Event (Simple Class)

Each event needs:
- when it happens
- what it does

In [63]:
class Event:
    def __init__(self, time):
        self.time = time

    def process(self, engine):
        raise NotImplementedError

Event Types

In [64]:
class OrderArrivalEvent(Event):
    def __init__(self, time, order):
        super().__init__(time)
        self.order = order

    def process(self, engine):
        engine.order_book.add_order(self.order)

In [65]:
class MarketCloseEvent(Event):
    def process(self, engine):
        engine.running = False

In [78]:
class SnapshotEvent:
    def __init__(self, time):
        self.time = time

    def process(self, engine):
        bb = engine.order_book.best_bid()
        ba = engine.order_book.best_ask()

        if bb is not None and ba is not None:
            engine.l1_snapshots.append({
                "time": engine.time,
                "best_bid": bb,
                "best_ask": ba,
                "spread": ba - bb,
                "mid": (ba + bb) / 2
            })

        depth = 3
        bids = [(o.price, o.qty) for _, _, o in heapq.nsmallest(depth, engine.order_book.bids)]
        asks = [(o.price, o.qty) for _, _, o in heapq.nsmallest(depth, engine.order_book.asks)]
        t = (bids, asks)
        engine.l2_snapshots.append(t)

Implement the MarketEngine

In [79]:
class MarketEngine:
    def __init__(self, order_book):
        self.time = 0
        self.event_queue = []
        self.seq = 0          # tie-breaker
        self.order_book = order_book
        self.running = True
        self.l1_snapshots = []
        self.l2_snapshots = []

    def push_event(self, event):
        heapq.heappush(
            self.event_queue,
            (event.time, self.seq, event)
        )
        self.seq += 1

    def run(self):
        while self.event_queue and self.running:
            event_time, _, event = heapq.heappop(self.event_queue)

            # advance simulation time
            self.time = event_time

            # process event
            event.process(self)

This satisfies:
- discrete time
- deterministic ordering
- priority queue scheduler
- no sleep
- clean stop on market close

USAGE

In [80]:
book = OrderBook()
engine = MarketEngine(book)

In [81]:
# Schedule ask ladder
engine.push_event(
    OrderArrivalEvent(1, Order(1, "sell", 101, 10, 1))
)
engine.push_event(
    OrderArrivalEvent(2, Order(2, "sell", 102, 20, 2))
)
engine.push_event(
    OrderArrivalEvent(3, Order(3, "sell", 103, 30, 3))
)

In [82]:
# Schedule market buy with latency
engine.push_event(
    OrderArrivalEvent(5, Order(4, "buy", float("inf"), 60, 5)) # In a discrete event simulation, time advances by jumping between scheduled event timestamps rather than progressing continuously.
)

In [83]:
# take snapshot every 1 second
SIMULATION_TIME = 10
for t in range(0, SIMULATION_TIME + 1):
    engine.push_event(SnapshotEvent(t))

In [84]:
# Schedule market close
# engine.push_event(MarketCloseEvent(10))

In [85]:
# run simulation
engine.run()

In [86]:
# latency simulation
send_time = 5
latency = 2
arrival_time = send_time + latency

engine.push_event(
    OrderArrivalEvent(arrival_time, order)
)

I implemented a discrete-event simulation using a priority queue scheduler. Events are processed in timestamp order, simulation time advances discretely, and latency is modeled by delayed order arrival events rather than wall-clock sleep

Day 4

The Tape (Trade Log)

In [92]:
tape_df = pd.DataFrame(book.tape)
print(tape_df.head())

           time  price  qty
0  1.767785e+09    101   10
1  1.767785e+09    102   20
2  1.767785e+09    103   30
3  1.767785e+09    101   10
4  1.767785e+09    102   20


L1 Snapshots (Best Bid / Ask Over Time)

At fixed times (e.g. every 1 second), record: time, best_bid, best_ask, spread, mid_price

In [88]:
l1_df = pd.DataFrame(engine.l1_snapshots)
print(l1_df.head())

Empty DataFrame
Columns: []
Index: []


L1 snapshots were initially empty because the order book never contained both bids and asks simultaneously. Adding resting liquidity on both sides resolved the issue.

In [89]:
# Add initial bid
engine.push_event(
    OrderArrivalEvent(0, Order(0, "buy", 100, 50, 0))
)

# Ask ladder
engine.push_event(
    OrderArrivalEvent(1, Order(1, "sell", 101, 10, 1))
)
engine.push_event(
    OrderArrivalEvent(2, Order(2, "sell", 102, 20, 2))
)
engine.push_event(
    OrderArrivalEvent(3, Order(3, "sell", 103, 30, 3))
)

# Market buy
engine.push_event(
    OrderArrivalEvent(5, Order(4, "buy", float("inf"), 60, 5))
)

for t in range(0, SIMULATION_TIME + 1):
    engine.push_event(SnapshotEvent(t+0.00001)) # Snapshots are scheduled slightly after order events at the same timestamp to ensure the recorded L1 state reflects all processed orders.

engine.run()

l1_df = pd.DataFrame(engine.l1_snapshots)
print(l1_df.head())

      time  best_bid  best_ask  spread    mid
0  1.00001       100       101       1  100.5
1  2.00001       100       101       1  100.5
2  3.00001       100       101       1  100.5
3  4.00001       100       101       1  100.5
4  7.00001       100       103       3  101.5


In [90]:
l2_df = pd.DataFrame(engine.l2_snapshots)
print(l2_df.head())

    0                                  1
0  []                                 []
1  []                        [(101, 10)]
2  []             [(101, 10), (102, 20)]
3  []  [(101, 10), (102, 20), (103, 30)]
4  []  [(101, 10), (102, 20), (103, 30)]


L2 snapshots capture the top few price levels on the bid and ask sides of the order book, along with their quantities, at fixed points in simulation time. Unlike L1 data, which records only the best bid and ask, L2 data provides a deeper view of available liquidity. In this simulation, L2 snapshots are logged passively for analysis and do not influence order matching or agent behavior with a depth of 3.

VWAP (Volume-Weighted Average Price) is computed only from the trade tape, since it represents execution prices weighted by traded volume.

In [94]:
vwap = (tape_df["price"] * tape_df["qty"]).sum() / tape_df["qty"].sum()
print(vwap)

102.33333333333333


Spread is computed only from L1 snapshots, using the difference between the best ask and best bid at each snapshot.

In [95]:
avg_spread = l1_df["spread"].mean()
print(avg_spread)

2.0


Mid-price volatility is computed from the time series of mid-prices, using returns derived from L1 snapshot data.

In [97]:
l1_df["log_return"] = np.log(l1_df["mid"]).diff()
mid_volatility = l1_df["log_return"].std()

Trade-price volatility (for comparison)

In [99]:
tape_df["log_return"] = np.log(tape_df["price"]).diff()
trade_volatility = tape_df["log_return"].std()
print(trade_volatility)

0.01315385028881364


The analytics pipeline is considered correct only if all the following conditions hold:
- VWAP lies between the session’s minimum and maximum trade prices
- Spread is non-negative at all recorded L1 snapshots
- Mid-price volatility is lower than trade-price volatility, since mid-price removes bid–ask bounce
- Re-running the simulation with the same random seed produces identical metrics

In [100]:
# VWAP must lie between session low and high
assert vwap >= tape_df["price"].min(), "VWAP below session low"
assert vwap <= tape_df["price"].max(), "VWAP above session high"

# Spread must never be negative
assert (l1_df["spread"] >= 0).all(), "Negative spread detected"

# Mid-price volatility should be lower than trade-price volatility
assert mid_volatility < trade_volatility, "Mid-price volatility too high"

print("✅ Day 4 analytics validation PASSED")

✅ Day 4 analytics validation PASSED
